In [7]:
# Set up Azure OpenAPI key 
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [8]:
api_key = "0140412acf0e4f69a00b0c63c454a2e8"
azure_endpoint = "https://thangtd.openai.azure.com/"
api_version = "2023-03-15-preview"
llm = AzureOpenAI(
    model="GPT35",
    deployment_name="gpt",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)# You need to deploy your own embedding model as well as your own chat completion model
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ADA002",
    api_key=api_key,
    azure_endpoint=azure_endpoint,
    api_version=api_version,
)

In [9]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [15]:
from llama_index.core.evaluation import DatasetGenerator, QueryResponseDataset
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.readers.file import FlatReader, HTMLTagReader
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SemanticSplitterNodeParser

from pathlib import Path

reader = SimpleDirectoryReader(input_dir="data_v2/cleaned_data")

docs = reader.load_data()

In [16]:
print(len(docs))

6


In [18]:
from llama_index.core.node_parser import HTMLNodeParser, SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_handler

from llama_index.core.node_parser import SemanticSplitterNodeParser


pipeline = IngestionPipeline(
    transformations=[
        HTMLNodeParser.from_defaults(),
        SemanticSplitterNodeParser(
            buffer_size=1, breakpoint_percentile_threshold=95, embed_model=embed_model
        ),
        embed_model,
    ],
)
nodes = pipeline.run(documents=docs)

INFO:httpx:HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://thangtd.openai.azure.com//openai/deployments/ADA002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 20

In [19]:
print(len(nodes))

19


In [20]:
# Indexing 
index = VectorStoreIndex(nodes)

In [21]:
# Store database
from llama_index.core import StorageContext, load_index_from_storage

index.storage_context.persist(persist_dir="database/")

In [22]:
# load database 
# from llama_index.core import StorageContext, load_index_from_storage

# # rebuild storage context
# storage_context = StorageContext.from_defaults(persist_dir="<persist_dir>")

# # load index
# index = load_index_from_storage(storage_context)
